In [6]:
import sys
import time
import pandas as pd
import numpy as np

# QCGPU Imports
import qcgpu

# ProjectQ Imports
from projectq import MainEngine
import projectq.ops as ops
from projectq.backends import Simulator
from projectq.types import Qureg



In [7]:
def bench_qcgpu(n, depth):
    state = qcgpu.State(n)

    h = qcgpu.gate.h()
    x = qcgpu.gate.x()
    sqrt_x = qcgpu.gate.sqrt_x()

    print('started')
    start = time.time()

    for level in range(depth):
        for q in range(n):
    
            state.apply_gate(h, q)
            state.apply_gate(sqrt_x, q)

            if q != 0:
                state.apply_controlled_gate(x, q, 0)
        
    runtime = time.time() - start
    print('ended: ', runtime)
    return runtime
    
def bench_projectq(n, depth):
    eng = MainEngine(backend=Simulator(gate_fusion=True), engine_list=[])
    qbits = eng.allocate_qureg(n)

    print('started')
    start = time.time()

    for level in range(depth):
        for q in qbits:
            ops.H | q
            ops.SqrtX | q
            if q != qbits[0]:
                ops.CNOT | (q, qbits[0])

    runtime = time.time() - start
    print('ended: ', runtime)

    for q in qbits:
        ops.Measure | q
    eng.flush()
    return runtime


In [8]:
bench_qcgpu(26,5)

started
('ended: ', 0.2940499782562256)


0.2940499782562256

In [9]:
bench_projectq(26,5)

started
('ended: ', 12.032409906387329)


12.032409906387329

In [13]:
raw_data = {
    'num_qubits': [2,2,2,3,3,3,4,4,4,5,5,5,6,6,6], 
    'depth': [10,20,30,10,20,30,10,20,30,10,20,30,10,20,30], 
    'time': [0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256, 0.2940499782562256]
}

df = pd.DataFrame(raw_data, columns = ['num_qubits', 'depth', 'time'])

df

df.to_csv('out.csv')